In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import chromedriver_binary
from pymongo import MongoClient 
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
% matplotlib inline

In [2]:
# making connection to mongodb

client = MongoClient()
database_schoolFinds = client['db_schoolFinds']
collection_schoolsFindsPass = database_schoolFinds['collection_schoolFindsPass']
collection_schoolsFindsFail = database_schoolFinds['collection_schoolFindsFail']

In [3]:
driver = webdriver.Chrome()
driver.get("http://www.schoolfinds.com/")
driver.maximize_window()

# finding all the cities links

driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
sleep(3)

selectCityButton = driver.find_element_by_xpath("//div[@class='header-content visible-desktop']//div[@class='span4']//a[2]")
selectCityButton.click()
sleep(2)

allCityLinks = driver.find_elements_by_xpath("//div[@class='modal-body']//ul[@class='last']//a")

closeButton = driver.find_element_by_xpath('//*[@id="popup"]/footer/button')
closeButton.click()
sleep(2)

In [4]:
all_city_links_list = []

for eachCity in allCityLinks:
    all_city_links_list.append(eachCity.get_attribute('href'))
    
all_city_links_list = all_city_links_list[150:200]

In [5]:
all_city_links_list

['http://www.schoolfinds.com/andhra-pradesh/hyderabad.html',
 'http://www.schoolfinds.com/andhra-pradesh/visakhapatnam.html',
 'http://www.schoolfinds.com/andhra-pradesh/secunderabad.html',
 'http://www.schoolfinds.com/andhra-pradesh/west-godavar.html',
 'http://www.schoolfinds.com/andhra-pradesh/guntur.html',
 'http://www.schoolfinds.com/andhra-pradesh/ranga-reddy.html',
 'http://www.schoolfinds.com/andhra-pradesh/vijayawada.html',
 'http://www.schoolfinds.com/andhra-pradesh/kurnool.html',
 'http://www.schoolfinds.com/andhra-pradesh/nellore.html',
 'http://www.schoolfinds.com/tamil-nadu/kanyakumari.html',
 'http://www.schoolfinds.com/tamil-nadu/tirunelveli.html',
 'http://www.schoolfinds.com/tamil-nadu/chennai.html',
 'http://www.schoolfinds.com/tamil-nadu/coimbatore.html',
 'http://www.schoolfinds.com/tamil-nadu/nilgiris.html',
 'http://www.schoolfinds.com/tamil-nadu/madras.html',
 'http://www.schoolfinds.com/tamil-nadu/tiruchirapalli.html',
 'http://www.schoolfinds.com/uttarakhand/n

In [6]:
for eachCity in all_city_links_list:
    
    
    print("working for URL ", eachCity)
    driver.get(eachCity)
    sleep(5)
    
    noOfSchoolsTotal = int(re.sub(r"\D", '', str(driver.find_element_by_xpath(r"//div[@class='product-count']").text)))  # 196
    print("Total no of schools are : ", noOfSchoolsTotal)
    
    noOfOlsEstimated = int(noOfSchoolsTotal/10)   # 19 suppose
    print("possible number of ols estimated : ", noOfOlsEstimated)
    
    print("scrolling to page end!!! : for ",noOfOlsEstimated, " times")
    
    for i in range(noOfOlsEstimated):   # cover ols created
        
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        sleep(3)
        
        
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
        
        
        
    if (noOfSchoolsTotal < 10):
        print("no of schools are less than 10")
        driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
    
        
        noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][1]" + "//li[contains(@class,'item')]"))
        print()
        print("No of lis  presentin this single ol :", noOfLisPresent_in_this_ol)
        
        for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"
            
            try:
                schoolAboutDict = {}

                schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                schoolAboutDict['schoolName'] = schoolName

                schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                schoolAboutDict['schoolAddress'] = schoolAddressRefined

                schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                schoolAboutDict['schoolEmail'] = schoolEmail

                schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                schoolAboutDict['schoolContact'] = schoolContact

                schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                schoolAboutDict['schoolURL'] = schoolURL
                
                if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                    print("Already Present in the db")
                    continue

                collection_schoolsFindsPass.insert_one(schoolAboutDict)
                sleep(2)
                
                print("School Details : ", schoolAboutDict)
                print()
                
            except:
                
                print("Error for the head url city : ", eachCity)
                
                schoolDictFail = {}
                schoolDictFail['Error City URL'] = str(eachCity)
                
                if len(str(schoolName) > 0):
                    schoolDictFail['Error School Name'] = str(schoolName)
                
                collection_schoolsFindsFail.insert_one(schoolDictFail)
                
                
        
    else:
        
        if (noOfSchoolsTotal > noOfOlsEstimated*10):   # 196 > 190 
            print("WE NEED AN EXTRAA ITERATION!!!!  AS  :", noOfSchoolsTotal," > " ,noOfOlsEstimated*10)
        
            for i in range(1, noOfOlsEstimated + 2):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19,20

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        else:
            print("schools can be exact multiples of 10 ")
            
            for i in range(1, noOfOlsEstimated + 1):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        

working for URL  http://www.schoolfinds.com/andhra-pradesh/hyderabad.html
Total no of schools are :  57
possible number of ols estimated :  5
scrolling to page end!!! : for  5  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 57  >  50
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'AZRA PUBLIC SCHOOL', 'schoolAddress': ': 10-2-510/1 ASIF NAGAR MEHDIPATNAM MAIN ROAD HYDERABAD ANDHRA PRADESH -500028', 'schoolEmail': 'Email: azra.school@gmail.com', 'schoolContact': 'Contact Number: 040 -3530453 3530262', 'schoolURL': 'http://www.schoolfinds.com/azra-public-school.html', '_id': ObjectId('5b89a22bedf2ee25c08e6528')}

School Inserted with success
School Details :  {'schoolName': 'HINDU PUBLIC SCHOOL', 'schoolAddress': ': sanath nagar hyderabad andhra pradesh -500018', 'schoolEmail': 'Email: hindu_pschool88@yahoo.com', 'schoolContact': 'Contact Number: 040 -23704355', 'sch


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'CMR INTERNATIONAL SCHOOL', 'schoolAddress': ': 173- Plassey Lane- Bowenpally SECUNDERABAD HYDERABAD BALANAGAR ANDHRA PRADESH -500011', 'schoolEmail': 'Email: cmrschool@ymail.com', 'schoolContact': 'Contact Number: 4027758892', 'schoolURL': 'http://www.schoolfinds.com/cmr-international-school.html', '_id': ObjectId('5b89a26dedf2ee25c08e653c')}

School Inserted with success
School Details :  {'schoolName': 'CHINMAYA VIDYALAYA', 'schoolAddress': ': KUNDAN BAGH BEGUM PET HYDERABAD ANDHRA PRADESH -500016', 'schoolEmail': 'Email: pvgrao@hotmail.com', 'schoolContact': 'Contact Number: 040 -3317594', 'schoolURL': 'http://www.schoolfinds.com/chinmaya-vidyalaya-18733.html', '_id': ObjectId('5b89a270edf2ee25c08e653d')}

School Inserted with success
School Details :  {'schoolName': 'CHAITANYA BHARATHI PUBLIC SCHOOL', 'schoolAddress': ': GANDIPET HYDERABAD ANDHRA PRADESH -500075', 'schoolEmail': 'Email

School Inserted with success
School Details :  {'schoolName': 'SILVER OAKS HIGH SCHOOL', 'schoolAddress': ': BACHUPALLY QUTHBULLAPUR MANDOL- RANGA REDDY DISTT. HYDERABAD Andhra Pradesh -500072', 'schoolEmail': 'Email: kayyany@md3.vsnl.net.in', 'schoolContact': 'Contact Number: 040 -23047777', 'schoolURL': 'http://www.schoolfinds.com/silver-oaks-high-school.html', '_id': ObjectId('5b89a2b1edf2ee25c08e6551')}

School Inserted with success
School Details :  {'schoolName': 'ROOTS MONTESSORI SCHOOL', 'schoolAddress': ': ROAD NO.6 BANJARA HILLS HYDERABAD ANDHRA PRADESH -500036', 'schoolEmail': 'Email: roots_school2000@yahoo.com', 'schoolContact': 'Contact Number: 040 -3350190', 'schoolURL': 'http://www.schoolfinds.com/roots-montessori-school.html', '_id': ObjectId('5b89a2b5edf2ee25c08e6552')}

School Inserted with success
School Details :  {'schoolName': 'SARTHI SCHOOL', 'schoolAddress': ': HABSIGUDA X ROAD OPP CCMB HYDERABAD ANDHRA PRADESH -500007', 'schoolEmail': 'Email: samarpanpublicscho

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': WALTAIR THATICHETLAPALEM VISAKHAPATNAM ANDHRA PRADESH -530016', 'schoolEmail': 'Email: kvwaltair@yahoo.com', 'schoolContact': 'Contact Number: 0891 -746539', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-108858.html', '_id': ObjectId('5b89a307edf2ee25c08e6565')}

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': MALKAPURAM GANDHIGRAM PO NAUSENABAGH VISAKHAPATNAM ANDHRA PRADESH -530005', 'schoolEmail': 'Email: pkvno2i@rediffmail.com', 'schoolContact': 'Contact Number: 0891 -577136', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-108863.html', '_id': ObjectId('5b89a30bedf2ee25c08e6566')}

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': INS KALINGA CHEPALA UPPADA POST BHEEMUNIPATNAM (VIA) VISAKHAPATNAM D ANDHRA PRADESH -531163', 'schoolEmail': 'Email: 

School Inserted with success
School Details :  {'schoolName': 'S R DIGI SCHOOL', 'schoolAddress': ': zinc smelter post visakhapatnam andhra pradesh -530015', 'schoolEmail': 'Email: aprabhakar@ymail.com', 'schoolContact': 'Contact Number: 0891 -2593594', 'schoolURL': 'http://www.schoolfinds.com/s-r-digi-school.html', '_id': ObjectId('5b89a34fedf2ee25c08e657a')}

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA NO.2', 'schoolAddress': ': NAUSENABAGH GANDHIGRAM (PO) VISAKHAPATNAM ANDHRA PRADESH -530005', 'schoolEmail': 'Email: kv2principal@yahoo.com', 'schoolContact': 'Contact Number: 0891 -577614', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-no-24810.html', '_id': ObjectId('5b89a353edf2ee25c08e657b')}

School Inserted with success
School Details :  {'schoolName': 'LITTLE ANGELS SCHOOL', 'schoolAddress': ': Sector 9 ukkungram visakhapatnam andhra pradesh -500032', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number:', 'schoolURL': 'ht

...

School Inserted with success
School Details :  {'schoolName': 'MODI PUBLIC SCHOOL', 'schoolAddress': ': DADABARI EXTENSION KOTA RAJASTHAN -324009', 'schoolEmail': 'Email: mps_kota@yahoo.com', 'schoolContact': 'Contact Number: 0744 -2505496', 'schoolURL': 'http://www.schoolfinds.com/modi-public-school.html', '_id': ObjectId('5b89afa5edf2ee25c08e6869')}

School Inserted with success
School Details :  {'schoolName': 'NALANDA ACADEMY', 'schoolAddress': ': ROAD NO 7 ANANTPURA KOTA RAJASTHAN -814142', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 0744 -2420963', 'schoolURL': 'http://www.schoolfinds.com/nalanda-academy.html', '_id': ObjectId('5b89afa7edf2ee25c08e686a')}

School Inserted with success
School Details :  {'schoolName': 'SARVODAYA SENIOR SECONDARY SCHOOL', 'schoolAddress': ': IPC-17- RIICO INSTITUTIONAL AREA RANPURA- KOTA- RAJASTHAN -324003', 'schoolEmail': 'Email: sarvodayacbseschool@gmail.com', 'schoolContact': 'Contact Number: 0744-2411229', 'schoolURL': 'http://ww

School Inserted with success
School Details :  {'schoolName': 'DAV PUBLIC SCHOOL', 'schoolAddress': ': SECTOR C TALWANDI KOTA RAJASTHAN -324005', 'schoolEmail': 'Email: davkota@gmail.com', 'schoolContact': 'Contact Number: 0744 -2405992', 'schoolURL': 'http://www.schoolfinds.com/dav-public-school-22096.html', '_id': ObjectId('5b89afebedf2ee25c08e687f')}

School Inserted with success
School Details :  {'schoolName': 'EMMANUEL MISSION SCHOOL', 'schoolAddress': ': C-Scheme- TALWANDI KOTA RAJASTHAN -324005', 'schoolEmail': 'Email: samuel@christforindia.com', 'schoolContact': 'Contact Number: 0744 -427339', 'schoolURL': 'http://www.schoolfinds.com/emmanuel-mission-school.html', '_id': ObjectId('5b89afeeedf2ee25c08e6880')}

School Inserted with success
School Details :  {'schoolName': 'GURU NANAK PUBLIC SCHOOL', 'schoolAddress': ': KOTA BUNDI ROAD BADGAON LADPURA KOTA RAJASTHAN -324008', 'schoolEmail': 'Email: stpaulscor@yahoo.co.in', 'schoolContact': 'Contact Number: 0744 -2371587', 'school

Total no of schools are :  24
possible number of ols estimated :  2
scrolling to page end!!! : for  2  times
WE NEED AN EXTRAA ITERATION!!!!  AS  : 24  >  20
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
School Inserted with success
School Details :  {'schoolName': 'ASSAM RIFFLES PUBLIC SCHOOL', 'schoolAddress': ': mantripukhri imphal manipur -795001', 'schoolEmail': 'Email: ananthijebasingh@vsnl.net', 'schoolContact': 'Contact Number: 05442 -282156', 'schoolURL': 'http://www.schoolfinds.com/assam-riffles-public-school.html', '_id': ObjectId('5b89b077edf2ee25c08e6894')}

School Inserted with success
School Details :  {'schoolName': 'MANIPUR PUBLIC SCHOOL', 'schoolAddress': ': KOIRENGEI MANTRIPUKHRI IMPHAL MANIPUR -795001', 'schoolEmail': 'Email: m_publicschool.co.in', 'schoolContact': 'Contact Number: 86123', 'schoolURL': 'http://www.schoolfinds.com/manipur-public-school.html', '_id': ObjectId('5b89b079edf2ee25c08e6895')}


School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': CRPF GROUP CENTRE LANGJING IMPHAL MANIPUR -795113', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 22362', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-108959.html', '_id': ObjectId('5b89b0b2edf2ee25c08e68a9')}

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': LEIMAKHONG FIELD POST OFFICE MANTIRI PUKHRI BPO KARAKHUL- IMPHAL MANIPUR -795002', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 05964 -232143', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-108960.html', '_id': ObjectId('5b89b0b5edf2ee25c08e68aa')}

School Inserted with success
School Details :  {'schoolName': 'ZENITH ACADEMY', 'schoolAddress': ': Sangaiprou- Imphal West- Manipur -795001', 'schoolEmail': 'Email: zenithacademyimphal@yahoo.com', 'schoolContact': 'Contact Number: 9862580059', 'schoolURL': 'http://www.schoo

Total no of schools are :  6
possible number of ols estimated :  0
scrolling to page end!!! : for  0  times
no of schools are less than 10

No of lis  presentin this single ol : 6
School Details :  {'schoolName': 'ACHARYA NARENDER DEO PUB SCHOOL', 'schoolAddress': ': AT & PO KHALPURA VIA GULTAINGANJ DISTT SARAN CHAPRA BIHAR -841211', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 06152 -73493', 'schoolURL': 'http://www.schoolfinds.com/acharya-narender-deo-pub-school.html', '_id': ObjectId('5b89b11cedf2ee25c08e68be')}

School Details :  {'schoolName': 'BHAGWAT VIDYAPEETH', 'schoolAddress': ': RUSSI CHHAWNI BHAGWAN BAZAR CHHAPRA SARAN BIHAR -841301', 'schoolEmail': 'Email: pgps96@yahoo.co.in', 'schoolContact': 'Contact Number: 06152 -232341', 'schoolURL': 'http://www.schoolfinds.com/bhagwat-vidyapeeth.html', '_id': ObjectId('5b89b11fedf2ee25c08e68bf')}

School Details :  {'schoolName': 'CHAPRA CENTRAL SCHOOL', 'schoolAddress': ': BEHIND GHOSH COLONY NEAR BAZAR SAMITI SARHA RO